In [11]:
# Installing essential libraries
!pip install -qU langchain openai weaviate-client tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.6 MB/s eta 0:00:00


In [43]:
# Importing essential modules

import os
import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.document_loaders import TextLoader
from langchain import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

In [27]:
# Storing API Keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
WEAVIATE_URL = getpass.getpass("WEAVIATE_URL:")
os.environ["WEAVIATE_API_KEY"] = getpass.getpass("WEAVIATE_API_KEY:")

OpenAI API Key:··········
WEAVIATE_URL:··········
WEAVIATE_API_KEY:··········


In [28]:
# Loading a text document and splitting into chunks

loader = TextLoader("resume.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [29]:
# Coverting the chunks into embeddings using OpenAI and storing them as weaviate cluster

embeddings = OpenAIEmbeddings()
db = Weaviate.from_documents(docs, embeddings, weaviate_url=WEAVIATE_URL, by_text=False)


In [37]:
# Creating a QA chain on the vectorstore

resume_qa = ConversationalRetrievalChain.from_llm(llm = OpenAI(temperature = 0),retriever = db.as_retriever(search_kwargs={"k": 1}))

In [41]:
# Ask a single query

chat_history = []

query = "What is this document about?"

result = resume_qa({'question' : query, 'chat_history': []})

print(result)

{'question': 'What is this document about?', 'chat_history': [], 'answer': ' This document is about a job opening for an HR Administrator/Marketing Associate.'}


In [44]:
# Creating a function to ask queries with stored chat history

def resume_retreiver():
  chat_history = []
  while True:
    with get_openai_callback() as cb:
      query = input("Ask your question: ")
      result = resume_qa({'question' : query, 'chat_history': chat_history})
      print(f'Total: {cb.total_tokens}, Prompt Tokens: {cb.prompt_tokens}, Completion Tokens: {cb.completion_tokens}')
      chat_history.append((query, result['answer']))
      if len(chat_history) > 5:
        chat_history.pop(0)
      print(result['answer'])

In [46]:
# chatbot on your document

resume_retreiver()

Ask your question: what is the document about?
Total: 87, Prompt Tokens: 71, Completion Tokens: 16
 This document is about a job opening for an HR Administrator/Marketing Associate.
Ask your question: tell me more
Total: 227, Prompt Tokens: 161, Completion Tokens: 66

This job opening is for a role that combines both HR administration and marketing. The job duties will likely include tasks related to both HR and marketing, such as recruiting, onboarding, developing marketing campaigns, and managing social media accounts.


KeyboardInterrupt: ignored